## Example

### Definicion de funciones necesarias para generacion malla

In [1]:
%config IPCompleter.greedy=True
import numpy as np
import os
import bempp.api
from mesh_converter import *
from derivate_operators import *

### Seleccion de proteina a simular, contenidas en carpetas pqr_files

In [2]:
# directory = '/home/ian/Desktop/Forces_bioelectrostatics' #Ubuntu
directory = 'C:\\Users\\ian\Desktop\\forces_calculation' #Windows
protein = 'arg'
forcefield = 'amber'
#dir_prot = directory+'/pqr_files/'+protein #Ubuntu
dir_prot = directory+'\\pqr_files\\'+protein #Windows
pf = protein +'_' + forcefield
density = 0.7
probe_radius = 1.4

#convert_pqr2xyzr('{}/{}.pqr'.format(dir_prot,pf),'{}/{}.xyzr'.format(dir_prot,pf))
#generate_nanoshaper_mesh('{}/{}.xyzr'.format(dir_prot,pf), dir_prot,pf,density,probe_radius,False)
grid = import_msms_mesh('{}/{}.face'.format(dir_prot,'arg_d04'),'{}/{}.vert'.format(dir_prot,'arg_d04'))

#grid2 = bempp.api.import_grid('/home/ian/Desktop/Forces_bioelectrostatics/5pti_d1.msh')


### Grafico de archivo mesh (Opcional)

In [3]:

bempp.api.PLOT_BACKEND = "gmsh"
grid.plot()

### Lectura de cargas contenidas en archivo .pqr

In [4]:
import numpy as np
q, x_q = np.array([]), np.empty((0,3))

ep_in = 4.
ep_ex = 80.
k = 0.125

# Read charges and coordinates from the .pqr file
molecule_file = open('{}/{}.pqr'.format(dir_prot,'arg'), 'r').read().split('\n')
for line in molecule_file:
    line = line.split()
    if len(line)==0 or line[0]!='ATOM': continue
    q = np.append( q, float(line[8]))
    x_q = np.vstack(( x_q, np.array(line[5:8]).astype(float) ))

### Right-hand side

With that, we can compute the potential due to the charges on the boundary, required for the right-hand side

In [5]:
# Function to calculate the potential by charges at boundary

dirichl_space = bempp.api.function_space(grid, "DP", 0)
neumann_space = bempp.api.function_space(grid, "DP", 0)

@bempp.api.real_callable
def charges_fun(x, n, domain_index, result):
    global q, x_q, ep_in
    suma = 0
    for k in range(len(q)):
        suma = suma + q[k]/(np.linalg.norm(x-x_q[k]))
    result[:] = suma/(4*np.pi*ep_in)
    #result[:] = np.sum(q / np.linalg.norm(x - x_q, axis=1))
    
charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun=charges_fun)
#charged_grid_fun.plot()

rhs = np.concatenate([charged_grid_fun.coefficients, 
                      np.zeros(neumann_space.global_dof_count)])


C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Operators and Matrix

Next, we generate the $2\times 2$ block matrix with the single and double layer operators of the Laplace and Yukawa kernels 

In [6]:
# Define Operators
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
identity = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
slp_in   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, k)
dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, k)

# Matrix Assembly
blocked = bempp.api.BlockedOperator(2, 2)
blocked[0, 0] = 0.5*identity + dlp_in
blocked[0, 1] = -slp_in
blocked[1, 0] = 0.5*identity - dlp_out
blocked[1, 1] = (ep_in/ep_ex)*slp_out
op_discrete = blocked.strong_form()

### Solver

We now use `gmres` from `scipy` to solve the system. 

In [7]:
import inspect
from scipy.sparse.linalg import gmres

array_it, array_frame, it_count = np.array([]), np.array([]), 0
def iteration_counter(x):
        global array_it, array_frame, it_count
        it_count += 1
        frame = inspect.currentframe().f_back
        array_it = np.append(array_it, it_count)
        array_frame = np.append(array_frame, frame.f_locals["resid"])
        #print "It: {0} Error {1:.2E}".format(it_count, frame.f_locals["resid"])        

x, info = gmres(op_discrete, rhs, callback=iteration_counter, tol=1e-3, maxiter=1000, restart = 2000)

print("The linear system was solved in {0} iterations".format(it_count))

The linear system was solved in 28 iterations


The two following `GridFunction` calls store the calculated boundary potential data (separated by $\phi$ and $\frac{\partial \phi}{\partial n}$) for visualization purposes. 

In [8]:
solution_dirichl = bempp.api.GridFunction(dirichl_space, 
                                          coefficients=x[:dirichl_space.global_dof_count])
solution_neumann = bempp.api.GridFunction(neumann_space, 
                                          coefficients=x[dirichl_space.global_dof_count:])
#solution_dirichl.plot()
#solution_neumann.plot()

### Energy calculaton

To compute $\phi_\text{reac}$ at the atoms locations, we use `operators.potential` to then multiply by the charge and add to compute $\Delta G_\text{solv}$. The `332.064` term is just a unit conversion constant.

In [10]:
slp_q = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
dlp_q = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
phi_q = slp_q*solution_neumann - dlp_q*solution_dirichl

cal2J = 4.184
qe = 1.60217646e-19
Na = 6.0221415e+23
ep_vacc = 8.854187818e-12
C0 = qe**2*Na*1e-3*1e10/(cal2J*ep_vacc)

# Reaction force calculation (qE)
h=0.001
grad_phi = solvent_potential_first_derivate(x_q, h, neumann_space, dirichl_space, solution_neumann, solution_dirichl)
F_reac = np.zeros([len(q),3])
for j in range(len(q)):
    F_reac[j,:] = -q[j]*grad_phi[j,:]
F_reactotal = np.zeros([3])
for j in range(len(q)):
    F_reactotal[:] = F_reactotal[:] + F_reac[j,:]
F_reactotal[:] = 4.184*4*np.pi*332.064*F_reactotal[:]
# Dielectric boundary force calculation (DBF)

grad_phi = solution_neumann.coefficients
f_dbf = np.zeros([grid.number_of_elements,3])
for j in range(grid.number_of_elements):
    f_dbf[j,:] = (grad_phi[j]**2)*grid.normals[j]*grid.volumes[j]
f_dbftotal = np.zeros([3])
for j in range(grid.number_of_elements):
    f_dbftotal[:] = f_dbftotal[:] + f_dbf[j,:]
f_dbftotal[:] = -4.184*0.5*(ep_ex-ep_in)*f_dbftotal[:]
#grid.number_of_elements
#grid.normals[1]
f_dbftotal
# Ionic boundary force calculation (IBF)
phi = solution_dirichl.coefficients
auxi = k*phi**2
f_ibf = np.zeros([grid.number_of_elements,3])
for j in range(grid.number_of_elements):
    f_ibf[j] = auxi[j]*grid.normals[j]*grid.volumes[j]
f_ibftotal = np.zeros([3])
for j in range(grid.number_of_elements):
    f_ibftotal[:] = f_ibftotal[:] + f_ibf[j,:]
f_ibftotal[:] = -4.184*0.5*(ep_ex-ep_in)*f_ibftotal[:]


    
print("Total reaction force: {:10.4f}{:10.4f}{:10.4f} [kcal/molA]".format(F_reactotal[0],F_reactotal[1],F_reactotal[2]))
print("Total dielectric boundary force: {:10.4f}{:10.4f}{:10.4f} [kcal/molA]".format(f_dbftotal[0],f_dbftotal[1],f_dbftotal[2]))
print("Total ionic boundary force: {:10.5f}{:10.5f}{:10.5f} [kcal/molA]".format(f_ibftotal[0],f_ibftotal[1],f_ibftotal[2]))
# total dissolution energy applying constant to get units [kcal/mol]
total_energy = 4.184*2*np.pi*332.064*np.sum(q*phi_q).real
print("Total solvation energy: {:7.2f} [kcal/Mol]".format(total_energy))

Total reaction force:    26.5827   -5.9984   10.9338 [kcal/molA]
Total dielectric boundary force:    -0.1283    0.0303   -0.0537 [kcal/molA]
Total ionic boundary force:   -0.00002  -0.00000  -0.00003 [kcal/molA]
Total solvation energy: -102.68 [kcal/Mol]


## References
[1] Yoon, B. J., & Lenhoff, A. M. (1990). A boundary element method for molecular electrostatics with electrolyte effects. *Journal of Computational Chemistry*, 11(9), 1080-1086.

[2] Dolinsky, T. J., Nielsen, J. E., McCammon, J. A., & Baker, N. A. (2004). PDB2PQR: an automated pipeline for the setup of Poisson–Boltzmann electrostatics calculations. *Nucleic acids research*, 32(suppl_2), W665-W667.

[3] Sanner, M. F., Olson, A. J., & Spehner, J. C. (1995, September). Fast and robust computation of molecular surfaces. In *Proceedings of the eleventh annual symposium on Computational geometry* (pp. 406-407). ACM.

[4] Sergio Decherchi and Walter Rocchia. A general and Robust Ray-Casting-Based
Algorithm for Triangulating Surfaces at the Nanoscale. PLOS ONE, 8(4):1–15, 2013.

[5] Timo Betcke and Matthew W. Scroggs. Bempp-cl: A fast python based just-in-time
compiling boundary element library. Journal of Open Source Software, 6(59):2879,
2021.

[6] S. Decherchi, A. Spitaleri, J. Stone and W. Rocchia, "NanoShaper-VMD interface: computing and visualizing surfaces, pockets and channels in
molecular systems". Bioinformatics, in press, 2018.